In [18]:
from mingus.containers import Composition, Track, Note, Bar, NoteContainer
from mingus.midi import fluidsynth

import mingus.core.chords as chords
import mingus.core.notes as notes
import mingus.core.scales as scales

import random
import time
import pandas as pd
import numpy as np

In [2]:
# Constants
MAX_TIME = 20
MAX_ROW_LEN = 88  # Picked as 88 keys, but arbitrary 
rule_30_states = {"111": '0', "110": '0', "101": '0', "000": '0', "100": '1', "011": '1', "010": '1', "001": '1'}

In [3]:
def create_elem_cell_auto_df(max_time, row_len, rule, initial_starts = None):
    df = pd.DataFrame(np.zeros((max_time, row_len)))

    for time, row in df.iterrows():
        if time == 0:
            if initial_starts is not None: 
                for start_ind in initial_starts:
                    row[start_ind] = 1
            else:
                # Initialize, depending on if the row length is even/odd
                start_ind = int(len(row) / 2)
                if len(row) % 2 == 0:
                    row[start_ind] = 1
                    row[start_ind-1] = 1
                else:
                    row[start_ind] = 1
                
            continue
        for ind in range(1, len(row)-1):
            key = str(int(df.loc[time-1][ind-1])) + str(int(df.loc[time-1][ind])) + str(int(df.loc[time-1][ind+1]))
            value = rule.get(key, 0)
            row[ind] = value

    return df

In [4]:
def print_elem_cell_auto(df):
    for time, row in df.iterrows():
        for r in row:
            if int(r) == 1:
                print(u'\u2588', end = '')
            else:
                print(' ', end = '')
        print('')

In [31]:
def piano_elem_cell_auto(df):
    for time_ind, row in df.iterrows():
        note_container = NoteContainer()
        i = 0
        for ind, r in enumerate(row):
            if int(r) == 1:
                if i >= 3:
                    continue
                note = Note()
                note.from_int(ind)
                i+=1
                
                note_container.add_notes(note)
        fluidsynth.play_NoteContainer(note_container)
        time.sleep(0.5)

In [35]:
def piano_filter_elem_cell_auto(df, filter_list: list):
    map_filter = set(filter_list)
    for time_ind, row in df.iterrows():
        for ind, r in enumerate(row):
            if int(r) == 1: # If it's a note
                note = Note()
                note.from_int(ind)
                
                if str(note.name) not in map_filter:
                    row[ind] = 0
                    
    return df

In [36]:
# Create the Cellular Automata
c = create_elem_cell_auto_df(MAX_TIME, MAX_ROW_LEN, rule_30_states)
print_elem_cell_auto(c)

                                           ██                                           
                                          ██ █                                          
                                         ██  ██                                         
                                        ██ ███ █                                        
                                       ██  █   ██                                       
                                      ██ ████ ██ █                                      
                                     ██  █    █  ██                                     
                                    ██ ████  █████ █                                    
                                   ██  █   ███     ██                                   
                                  ██ ████ ██  █   ██ █                                  
                                 ██  █    █ ████ ██  ██                                 
                     

In [37]:
# https://dmitri.mycpanel.princeton.edu/whatmakesmusicsoundgood.html
# Filter the Cellular Automata by scales
notes_filter = scales.Major("C").ascending()
print(notes_filter)
c = piano_filter_elem_cell_auto(c, notes_filter)
print_elem_cell_auto(c)

['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C']
                                           █                                            
                                           █ █                                          
                                         █   █                                          
                                        ██ █ █ █                                        
                                        █  █   ██                                       
                                      █  █ █   █                                        
                                      █  █        █                                     
                                    █   ██   █ ██                                       
                                   ██      █ █      █                                   
                                   █  █ █  █      █  █                                  
                                 █           █ █  █  █               

In [41]:
def convert_piano_to_dict(df):
    piano_dict = {}
    for time_ind, row in df.iterrows():
        notes = []
        for ind, r in enumerate(row):
            if int(r) == 1: # If it's a note
                notes.append(ind)
        piano_dict[time_ind] = notes
        
    return piano_dict

In [42]:
r = convert_piano_to_dict(c)

In [43]:
print(r)

{0: [43], 1: [43, 45], 2: [41, 45], 3: [40, 41, 43, 45, 47], 4: [40, 43, 47, 48], 5: [38, 41, 43, 47], 6: [38, 41, 50], 7: [36, 40, 41, 45, 47, 48], 8: [35, 36, 43, 45, 52], 9: [35, 38, 40, 43, 50, 53], 10: [33, 45, 47, 50, 53], 11: [33, 35, 36, 38, 41, 52, 53, 55], 12: [31, 35, 40, 41, 45, 48, 55], 13: [31, 33, 35, 36, 38, 43, 47, 48, 52, 55, 57], 14: [29, 33, 38, 40, 41, 50, 57], 15: [28, 29, 31, 33, 38, 40, 43, 45, 47, 48, 52, 53, 55, 57, 59], 16: [28, 31, 35, 36, 40, 41, 43, 45, 50, 52, 59, 60], 17: [26, 29, 31, 35, 38, 40, 45, 50, 53, 59], 18: [26, 29, 36, 38, 41, 45, 47, 48, 52, 53, 57, 62], 19: [24, 28, 29, 33, 36, 40, 41, 43, 47, 55, 57, 59, 60]}


In [69]:
# Voice Leading
def piano_voice_leading(notes):
    random_threshold = random.randint(2,5)
    direction = +1
    
    ref_notes = []
    end = len(notes)
    print(end)
    dict_notes = {}
    for time_ind, row in notes.items():
        print(time_ind, row)
        if time_ind == 0:
            ref_notes = row
            continue
        if random_threshold == 0:
            direction = +1 if direction == -1 else -1
            random_threshold = random.randint(2,5)
        
        new_row = []
        for ref_note in ref_notes:
            for pointed_note in row:
                if direction == +1:
                    # Find the next note up for all previous notes
                    if pointed_note > ref_note:
                        new_row.append(pointed_note)
                        continue
                else:
                    if pointed_note < ref_note:
                        new_row.append(pointed_note)
                        continue
        dict_notes[time_ind] = new_row
        random_threshold-=1
            
    return dict_notes

In [70]:
s = piano_voice_leading(r)

20
0 [43]
1 [43, 45]
2 [41, 45]
3 [40, 41, 43, 45, 47]
4 [40, 43, 47, 48]
5 [38, 41, 43, 47]
6 [38, 41, 50]
7 [36, 40, 41, 45, 47, 48]
8 [35, 36, 43, 45, 52]
9 [35, 38, 40, 43, 50, 53]
10 [33, 45, 47, 50, 53]
11 [33, 35, 36, 38, 41, 52, 53, 55]
12 [31, 35, 40, 41, 45, 48, 55]
13 [31, 33, 35, 36, 38, 43, 47, 48, 52, 55, 57]
14 [29, 33, 38, 40, 41, 50, 57]
15 [28, 29, 31, 33, 38, 40, 43, 45, 47, 48, 52, 53, 55, 57, 59]
16 [28, 31, 35, 36, 40, 41, 43, 45, 50, 52, 59, 60]
17 [26, 29, 31, 35, 38, 40, 45, 50, 53, 59]
18 [26, 29, 36, 38, 41, 45, 47, 48, 52, 53, 57, 62]
19 [24, 28, 29, 33, 36, 40, 41, 43, 47, 55, 57, 59, 60]


In [71]:
s

{1: [45],
 2: [45],
 3: [40, 41],
 4: [40],
 5: [38, 41],
 6: [38, 41],
 7: [45, 47, 48],
 8: [45, 52],
 9: [50, 53],
 10: [45, 47, 50, 53],
 11: [52, 53, 55],
 12: [31, 35, 40, 41],
 13: [31, 33, 35, 36, 38],
 14: [29, 33, 38, 40, 41],
 15: [28, 29, 31, 33, 38, 40],
 16: [45, 50, 52, 59, 60],
 17: [45, 50, 53, 59],
 18: [45, 47, 48, 52, 53, 57, 62],
 19: [47, 55, 57, 59, 60]}